In [1]:
import os

models_folder = "models"
print("Contents of models folder:", os.listdir(models_folder))

Contents of models folder: ['1.keras', '2.keras']


In [ ]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import nest_asyncio

nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware
origins = ["http://localhost", "http://localhost:3000"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load the trained model
MODEL = tf.keras.models.load_model("models/2.keras")

# Define class names
CLASS_NAMES = ["Early Blight", "Late Blight", "Healthy"]

@app.get("/ping")
async def ping():
    return "Hello, I am alive"

# Helper function to read the uploaded image
def read_file_as_image(data) -> np.ndarray:
    image = np.array(Image.open(BytesIO(data)))
    return image

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        # Read and preprocess the image
        image = read_file_as_image(await file.read())
        img_batch = np.expand_dims(image, 0)

        # Make prediction
        predictions = MODEL.predict(img_batch)
        predicted_class = CLASS_NAMES[np.argmax(predictions[0])]
        confidence = np.max(predictions[0])

        # Return the result
        return {
            "class": predicted_class,
            "confidence": float(confidence),
        }
    except Exception as e:
        return {"error": str(e)}

@app.get("/", response_class=HTMLResponse)
async def serve_html():
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Plant Disease Detection</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                background: #f8f8f8;
                text-align: center;
                padding: 50px;
            }
            h1 {
                color: #2d3436;
                font-size: 36px;
                margin-bottom: 20px;
            }
            form {
                display: inline-block;
                background: #fff;
                padding: 40px;
                border-radius: 15px;
                box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
            }
            input[type="file"] {
                padding: 10px;
                border-radius: 5px;
                background: #ffeaa7;
                border: 1px solid #fdcb6e;
                cursor: pointer;
                font-size: 16px;
            }
            button {
                margin-top: 15px;
                background-color: #00cec9;
                color: white;
                border: none;
                padding: 12px 30px;
                font-size: 18px;
                border-radius: 5px;
                cursor: pointer;
            }
            button:hover {
                background-color: #0984e3;
            }
            #result {
                margin-top: 20px;
                padding: 15px;
                background-color: #dfe6e9;
                border-radius: 10px;
                font-size: 20px;
                color: #2d3436;
                font-weight: bold;
            }
            .loading {
                color: #0984e3;
                font-size: 18px;
            }
        </style>
    </head>
    <body>
        <h1>🌿 poatato leaf Disease Detection</h1>
        <form id="upload-form">
            <input type="file" id="file" name="file" accept="image/*" required><br><br>
            <button type="submit">Predict</button>
        </form>
        <div id="result"></div>

        <script>
            const form = document.getElementById('upload-form');
            const resultDiv = document.getElementById('result');

            form.addEventListener('submit', async (e) => {
                e.preventDefault();

                const fileInput = document.getElementById('file');
                const file = fileInput.files[0];

                if (!file) {
                    resultDiv.innerHTML = "❗ Please select a file.";
                    return;
                }

                const formData = new FormData();
                formData.append("file", file);

                resultDiv.innerHTML = "<div class='loading'>Predicting...</div>";

                try {
                    const response = await fetch("/predict?" + new Date().getTime(), {
                        method: "POST",
                        body: formData,
                    });

                    const data = await response.json();
                    console.log("Response data:", data);

                    if (data.error) {
                        resultDiv.innerHTML = `❌ Error: ${data.error}`;
                    } else {
                        resultDiv.innerHTML = `
                            🌿 <strong>Prediction:</strong> ${data.class}<br>
                            📊 <strong>Confidence:</strong> ${(data.confidence * 100).toFixed(2)}%
                        `;
                    }
                } catch (error) {
                    resultDiv.innerHTML = `❌ An error occurred: ${error.message}`;
                }
            });
        </script>
    </body>
    </html>
    """
    return html_content


if __name__ == "__main__":
    uvicorn.run(app, host="localhost", port=8000)


INFO:     Started server process [9484]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
